## Setup

In [52]:
import QuantLib as ql
import numpy as np

In [3]:
termStructure=ql.RelinkableYieldTermStructureHandle()

startYears = 1
length = 5
type_ = ql.VanillaSwap.Payer
nominal = 1000.0
settlementDays = 2
fixedConvention = ql.Unadjusted
floatingConvention = ql.ModifiedFollowing
fixedFrequency = ql.Annual
floatingFrequency = ql.Semiannual
fixedDayCount = ql.Thirty360(ql.Thirty360.BondBasis)
index = ql.as_iborindex(ql.Euribor6M(termStructure))
calendar = index.fixingCalendar()
today = calendar.adjust(ql.Date.todaysDate())
settlement = calendar.advance(today,settlementDays,ql.Days)

In [4]:
def makeSwap(fixedRate):
    start = calendar.advance(settlement, startYears, ql.Years)
    maturity = calendar.advance(start, length, ql.Years)
    fixedSchedule = ql.Schedule(start, maturity,
                                ql.Period(fixedFrequency),
                                calendar,
                                fixedConvention,
                                fixedConvention,
                                ql.DateGeneration.Forward, False)
    
    floatSchedule = ql.Schedule(start, maturity,
                                ql.Period(floatingFrequency),
                                calendar,
                                floatingConvention,
                                floatingConvention,
                                ql.DateGeneration.Forward, False)
    
    swap = ql.VanillaSwap(type_, nominal,
                          fixedSchedule, fixedRate, fixedDayCount,
                          floatSchedule, index, 0.0,
                          index.dayCounter()
                          )
    
    swap.setPricingEngine(ql.DiscountingSwapEngine(termStructure))
    return swap

## Test Cached Values

In [5]:
usingAtParCoupons = ql.IborCoupon.usingAtParCoupons()

In [10]:
today = ql.Date(15, ql.February, 2002)
ql.Settings.instance().evaluationDate = today
settlement = ql.Date(19, ql.February, 2002)

In [12]:
termStructure.linkTo(ql.FlatForward(settlement, ql.QuoteHandle(ql.SimpleQuote(0.04875825)), ql.Actual365Fixed()))

In [13]:
atmRate = makeSwap(0.0).fairRate()

In [15]:
itmSwap = makeSwap(0.8*atmRate)
atmSwap = makeSwap(atmRate)
otmSwap = makeSwap(1.2*atmRate)

In [16]:
a = 0.048696
sigma = 0.0058904

In [17]:
model = ql.HullWhite(termStructure, a, sigma)

In [18]:
exerciseDates = []

In [19]:
leg = atmSwap.fixedLeg()

In [21]:
for cf in leg:
    coupon = ql.as_coupon(cf)
    exerciseDates+=[coupon.accrualStartDate()]

In [22]:
exercise = ql.BermudanExercise(exerciseDates)

In [23]:
treeEngine = ql.TreeSwaptionEngine(model, 50)

In [24]:
fdmEngine = ql.FdHullWhiteSwaptionEngine(model)

In [25]:
itmValue=0
atmValue=0
otmValue=0
itmValueFdm=0
atmValueFdm=0
otmValueFdm=0

In [26]:
if not usingAtParCoupons:
    itmValue    = 42.2402
    atmValue = 12.9032
    otmValue = 2.49758;
    itmValueFdm = 42.2111
    atmValueFdm = 12.8879
    otmValueFdm = 2.44443
else:
    itmValue    = 42.2460
    atmValue = 12.9069
    otmValue = 2.4985
    itmValueFdm = 42.2091
    atmValueFdm = 12.8864
    otmValueFdm = 2.4437
    

In [27]:
tolerance = 1.0e-4

In [32]:
swaption = ql.Swaption(itmSwap, exercise)
swaption.setPricingEngine(treeEngine)
print("Tree ITM: ",swaption.NPV(), " ", itmValue)

Tree:  42.24707523189427   42.246


In [34]:
swaption.setPricingEngine(fdmEngine)
print("FDM ITM:", swaption.NPV(), " ", itmValueFdm)

FDM: 42.209126374646075   42.2091


In [35]:
swaption = ql.Swaption(atmSwap, exercise)
swaption.setPricingEngine(treeEngine)
print("Tree ATM: ",swaption.NPV(), " ", atmValue)

Tree ATM:  12.882589156500163   12.9069


In [36]:
swaption.setPricingEngine(fdmEngine)
print("FDM ATM:", swaption.NPV(), " ", atmValueFdm)

FDM ATM: 12.886404686901107   12.8864


In [37]:
swaption = ql.Swaption(otmSwap, exercise)
swaption.setPricingEngine(treeEngine)
print("Tree OTM: ",swaption.NPV(), " ", otmValue)

Tree OTM:  2.4769278252589424   2.4985


In [38]:
swaption.setPricingEngine(fdmEngine)
print("FDM OTM:", swaption.NPV(), " ", otmValueFdm)

FDM OTM: 2.4437590288178934   2.4437


In [42]:
exerciseDates_adjusted = []
for exerciseDate in exerciseDates:
    exerciseDate = calendar.adjust(exerciseDate - 10)
    exerciseDates_adjusted+=[exerciseDate]
exercise = ql.BermudanExercise(exerciseDates_adjusted)

In [40]:
if not usingAtParCoupons:
    itmValue = 42.1791
    atmValue = 12.7699
    otmValue = 2.4368
else:
    itmValue = 42.1849
    atmValue = 12.7736
    otmValue = 2.4379
    

In [43]:
swaption = ql.Swaption(itmSwap, exercise)
swaption.setPricingEngine(treeEngine)
print("Tree ITM: ",swaption.NPV(), " ", itmValue)

Tree ITM:  42.19741129870393   42.1849


In [46]:
swaption = ql.Swaption(atmSwap, exercise)
swaption.setPricingEngine(treeEngine)
print("Tree ATM: ",swaption.NPV(), " ", atmValue)

Tree ATM:  12.782497212625326   12.7736


In [47]:
swaption = ql.Swaption(otmSwap, exercise)
swaption.setPricingEngine(treeEngine)
print("Tree OTM: ",swaption.NPV(), " ", otmValue)

Tree OTM:  2.4398613359077013   2.4379


## Test cached G2 values

In [48]:
today = ql.Date(15, ql.September, 2016)
ql.Settings.instance().evaluationDate = today
settlement = ql.Date(19, ql.September, 2016)

In [49]:
#flat yield term structure impling 1x5 swap at 5%
termStructure.linkTo(ql.FlatForward(settlement, ql.QuoteHandle(ql.SimpleQuote(0.04875825)), ql.Actual365Fixed()))

In [50]:
atmRate = makeSwap(0.0).fairRate()

In [57]:
swaptions = []
for s in np.arange(0.5,1.51,0.25):
    swap = makeSwap(s*atmRate)
    exerciseDates = []
    for cf in swap.fixedLeg():
        exerciseDates+=[ql.as_coupon(cf).accrualStartDate()]
    swaptions+=[ql.Swaption(swap,ql.BermudanExercise(exerciseDates))]

In [59]:
a=0.1
sigma=0.01
b=0.2
eta=0.013
rho=-0.5

In [60]:
g2Model = ql.G2(termStructure,a,sigma,b,eta,rho)

In [61]:
fdmEngine = ql.FdG2SwaptionEngine(g2Model,50,75,75,0,1e-3)

In [62]:
treeEngine = ql.TreeSwaptionEngine(g2Model, 50)

In [63]:
for i,swaption in enumerate(swaptions):
    swaption.setPricingEngine(fdmEngine)
    print (i, " FDM ", swaption.NPV())
    swaption.setPricingEngine(treeEngine)
    print(i, " Tree ", swaption.NPV())

0  FDM  103.22719265435109
0  Tree  103.25601669881533
1  FDM  54.650188390762885
1  Tree  54.67257297841144
2  FDM  20.046918192760035
2  Tree  20.142866748623074
3  FDM  5.2692418670892724
3  Tree  5.40639929778375
4  FDM  1.070928067209113
4  Tree  1.1067736710745917


## Time Snapping

In [64]:
today = ql.Date(8, ql.July, 2021)
ql.Settings.instance().evaluationDate = today

In [65]:
termStructure.linkTo(ql.FlatForward(settlement, ql.QuoteHandle(ql.SimpleQuote(0.02)), ql.Actual365Fixed()))

In [66]:
index=ql.Euribor3M(termStructure)

In [96]:
def makeBermudanSwaption(callDate):
    effectiveDate = ql.Date(15, ql.May, 2025)
    swap = ql.MakeVanillaSwap(ql.Period(10,ql.Years), index, 0.05,ql.Period(0,ql.Days),effectiveDate=effectiveDate,Nominal=10000.0,swapType=ql.VanillaSwap.Payer)
    exerciseDates = [effectiveDate,callDate]
    bermudanExercise = ql.BermudanExercise(exerciseDates)
    bermudanSwaption=ql.Swaption(swap,bermudanExercise)
    return bermudanSwaption

In [98]:
intervalOfDaysToTest = 10

In [110]:
for i in range(-intervalOfDaysToTest, intervalOfDaysToTest+1):
    initialCallDate = ql.Date(15, ql.May, 2030)
    calendar = index.fixingCalendar()
    callDate = initialCallDate + ql.Period(i, ql.Days)
    print(callDate)
    if (calendar.isBusinessDay(callDate)):
        bermudanSwaption = makeBermudanSwaption(callDate)
        model=ql.HullWhite(termStructure)
        bermudanSwaption.setPricingEngine(ql.FdHullWhiteSwaptionEngine(model))
        npvFD = bermudanSwaption.NPV()
        #print(npvFD)
        timesteps = 14 * 4 * 4
        bermudanSwaption.setPricingEngine(ql.TreeSwaptionEngine(model,timesteps))
        npvTree = bermudanSwaption.NPV()
        print("FD: ",npvFD, ", Tree: ",npvTree)
    

May 5th, 2030
May 6th, 2030
FD:  15.550026720814305 , Tree:  16.251837637775424
May 7th, 2030
FD:  15.567241445369966 , Tree:  16.341169405720102
May 8th, 2030
FD:  15.584463005944805 , Tree:  16.411464281913577
May 9th, 2030
FD:  15.60169140528961 , Tree:  16.473877204481585
May 10th, 2030
FD:  15.61892664628225 , Tree:  16.530069484171005
May 11th, 2030
May 12th, 2030
May 13th, 2030
FD:  15.6706734465037 , Tree:  16.6638185869186
May 14th, 2030
FD:  15.687936081597728 , Tree:  16.69756759540341
May 15th, 2030
FD:  15.70520557215797 , Tree:  16.726264896239694
May 16th, 2030
FD:  34.49628735416274 , Tree:  16.648528197447657
May 17th, 2030
FD:  34.52897740728422 , Tree:  16.570822784967348
May 18th, 2030
May 19th, 2030
May 20th, 2030
FD:  34.62711129664287 , Tree:  16.353334177579402
May 21st, 2030
FD:  34.659843854482745 , Tree:  16.294770799593312
May 22nd, 2030
FD:  34.692587051810996 , Tree:  16.226269505422437
May 23rd, 2030
FD:  34.725340894095 , Tree:  35.659404444984105
May 24